In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('../datasets/tong_hop1.csv')
df2 = pd.read_csv('../datasets/tong_hop.csv')
df2 = df2[~df2.label.isna()]

df2.set_index('PID', inplace=True)
df1.set_index('PID', inplace=True)

df1.index = df1.index.astype(str)
df2.index = df2.index.astype(str)

df1.shape, df2.shape

((23121, 18), (11255, 21))

In [3]:
intersec_pid = set(df1.index) & set(df2.index)

In [4]:
df3 = df2[~df2.index.isin(intersec_pid)]

In [5]:
print((df2.label !=1).sum())

2991


In [6]:
import pickle

In [7]:
with open('../models/ensemble_model_rf_knn_xgb_0705.pkl', 'rb') as f:
    model = pickle.load(f)

ModuleNotFoundError: No module named 'xgboost'